In [1]:
import Pkg; Pkg.add("Graphs")
using JuMP, Gurobi, Graphs, Plots, DataFrames, Random, Printf, LinearAlgebra, Distributions, Distances
const GRB_ENV = Gurobi.Env()

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18


Gurobi.Env(Ptr{Nothing} @0x0000000167224400, false, 0)

# Generate Data

In [16]:
#matching I (time), J (space, time)
struct Shipment #demand
    # Time slot each shipment take as KxT matrix
    residence_time::Matrix
    product_size::Int # product:shipment = 1:1 for now, can be M:1 i.e. shipment allocated to many locations
end

struct Warehouse #supply
    grid_size::Int
    row::Int
    col::Int
    distance_to_arrival::Matrix{Float64}
    distance_to_departure::Matrix{Float64}
    capacity::Matrix{Int}
    time_horizon::Int
end

In [18]:
G = 2
R = 2 #20
C = 2 #50
T = 10 #20
max_product_size = 10

10

## supply side

In [19]:
function WarehouseData(grid_size, warehouse_row, warehouse_col, max_product_size, time_horizon)
    
    arrival_location = (0, 0)
    departure_location = (warehouse_row, 0)

    locations = [(i*grid_size,j*grid_size) for i =1:warehouse_row, j = 1:warehouse_col]
    distance_to_arrival = [norm(arrival_location .- locations[i,j]) for i =1:warehouse_row, j = 1:warehouse_col]
    distance_to_departure = [norm(arrival_location .- locations[i,j]) for i =1:warehouse_row, j = 1:warehouse_col]

    capacity = rand(max_product_size:max_product_size, warehouse_row, warehouse_col) #very large to first observe the tightness of other constraints without capacity
    return Warehouse(
        grid_size,
        warehouse_row,
        warehouse_col,
        distance_to_arrival,
        distance_to_departure,
        capacity,
        time_horizon
    )
end
supply_data = WarehouseData(G, R, C, T)

Warehouse(2, 2, 2, [2.8284271247461903 4.47213595499958; 4.47213595499958 5.656854249492381], [2.8284271247461903 4.47213595499958; 4.47213595499958 5.656854249492381], [10 10; 10 10], 10)

## demand side

In [23]:
K = 6
rand_shipment_data = rand(1:T, (K, 2))
arrivals = minimum(rand_shipment_data, dims=2)
departures = maximum(rand_shipment_data, dims=2)

6×1 Matrix{Int64}:
  4
  9
  6
  3
  1
 10

In [ ]:
struct Shipment #demand
    # Time slot each shipment take as KxT matrix
    time_slots::Matrix
    product_size::Int # product:shipment = 1:1 for now, can be M:1 i.e. shipment allocated to many locations
end

function ShipmentData(time_horizon, max_product_size)
    K = 5 #num_shipments, 100
    rand_shipment_data = rand(1:T, (K, 2))
    arrivals = minimum(rand_shipment_data, dims=2)
    departures = maximum(rand_shipment_data, dims=2)

    rand_sizes = rand(1:max_product_size, K)
    shipments = Array{Shipment,1}(undef, K)

    for k in 1:K
        arrival_time = arrivals[k]
        departure_time = departures[k]
        product_size = rand_sizes[k]

        # Can't arrive and depart at the same time
        if arrival_time == departure_time
            if departure_time == T
                arrival_time -= 1
            else
                departure_time += 1
            end
        end
        shipments[k] = Shipment(arrival_time, departure_time, product_size)
    end
    # given arrival and departure time, impute 
    return Shipment(residence_time, product_size)
end
shipment_data = ShipmentData(max_product_size)

TBC

plot of space by time graph with shipment as a horizontal line from shipment to arrival at designated location

In [ ]:
function plot_from_data(
    data::Shipment, Warehouse,
)
    """Given a problem instance, plot it.
    """
    N = length(shipments)
    pl = plot(
        aspect_ratio = :equal, 
        # size = (500,500), 
        xlim = (0, 100), 
        ylim = (0, 100),
    )
    for i in 1:N
        annotate!(
            data.locations[i,1] + 2, 
            data.locations[i,2] + 2, 
            text("$i", 8)
        )
    end
    scatter!(
        data.locations[1:N, 1], 
        data.locations[1:N, 2], 
        label = "Customers",
    )
    scatter!(
        data.locations[[N+1], 1], 
        data.locations[[N+1], 2], 
        label = "Depot"
    )
    scatter!(
        legend = :outerright,
    )
    return pl
end
plot_from_data(data_large)

In [ ]:
function assign_loc(
    data::Shipment,
    
    """Given a problem instance, plot it.
    """

# Build Model

In [31]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60);

# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    x[1:K,1:S,1:T] >= 0, Bin
) ;

@objective(
    model,
    Min,
    sum(
        (distance_to_arrival[s] + distance_to_departure[s]) / (shipments[k].departure_time - shipments[k].arrival_time) * x[k,s,t]
        for k in 1:K
            s in 1:S,
            t in 1:T
    )
);

# Each product must be assigned to exactly one location during its time window:
for k in 1:K
    for t in shipments[k].arrival_time:shipments[k].departure_time
        @constraint(
            model,
            sum(
                x[k,s,t]
                for s in 1:S
            ) == 1
        );
    end
end

# Each product is only assigned to a location during its time window:
for k in 1:K
    for t in 1:shipments[k].arrival_time-1
        @constraint(
            model,
            sum(
                x[k,s,t]
                for s in 1:S
            ) == 0
        );
    end
end
for k in 1:K
    for t in shipments[k].departure_time+1:T
        @constraint(
            model,
            sum(
                x[k,s,t]
                for s in 1:S
            ) == 0
        );
    end
end

# Each location can hold at most one product at any specific time:
for s in 1:S
    for t in 1:T
        @constraint(
            model,
            sum(
                x[k,s,t]
                for k in 1:K
            ) <= 1
        );
    end
end

# Each location can only hold items up to max size of the location
for s in 1:S
    for t in 1:T
        for k in 1:K
            @constraint(
                model,
                x[k,s,t] * shipments[k].product_size <= location_max_size[s]
            );
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


UndefVarError: UndefVarError: s not defined

In [10]:
optimize!(model)

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2022000 rows, 2000000 columns and 6000000 nonzeros
Model fingerprint: 0x1ffe0ab3
Variable types: 0 continuous, 2000000 integer (2000000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 18136.769630
Presolve removed 2020943 rows and 1966192 columns
Presolve time: 4.34s
Presolved: 1057 rows, 33808 columns, 67616 nonzeros
Found heuristic solution: objective 6210.9207459
Variable types: 0 continuous, 33808 integer (33808 binary)
Found heuristic solution: objective 6107.1689755

Root relaxation: objective 5.915226e+03, 517 iterations, 0.02 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bo

## 

## Tighter formulation
Using binary variable z_{sl} which indicates shipment s is placed at location l is added to the fomultion to prevent bumping, meaning products are moved from one place to other for certain number of periods and come back (?). This may happen as the previous matrix which is S by T is aggregated over location so we don't know exactly where the shipments are stored at time T. Under the hood bumping may happen as, for instance longer staying thing is moved 

Tbh, I am confused whether z is needed but let's conclude by comparing the objective value. If it is higher than 5.915226451326e+03, then it's the proof feasible space contracted, meanign bumping happened previously.


In [13]:
for k in 1:K
    if start_time -1 < shipments[k].arrival_time
        indicator_arr2loc[k] = 1
    end
    if shipments[k].departure_time < end_time
        indicator_loc2dep[k] = 1
    end
end

indicator_loc2dep

UndefVarError: UndefVarError: start_time not defined

In [ ]:
model_tight = Model(Gurobi.Optimizer)
set_optimizer_attribute(model_tight, "TimeLimit", 60);

# r: Are we assigning the product from shipment s to location l at time t
# x: Are we assigning the product from shipment s to location l
@variable(
    model_tight,
    r[1:S,1:S,1:T] >= 0, Bin
    x[1:S,1:S] >= 0, Bin
) ;


@objective(
    model_tight,
    Min,
    sum(
        (distance_to_arrival[s] + distance_to_departure[s]) / (shipments[k].departure_time - shipments[k].arrival_time) * x[k,s,t]
        for k in 1:K
            s in 1:S,
            t in 1:T
    )
);

# Each product must be assigned to exactly one location during its time window:
for k in 1:K
    for t in shipments[k].arrival_time:shipments[k].departure_time
        @constraint(
            model_tight,
            sum(x[k,s,t] for s in 1:S) == 1
        );
    end
end

# Each product is only assigned to a location during its time window:
for k in 1:K
    for t in 1:shipments[k].arrival_time-1
        @constraint(
            model_tight,
            sum(x[k,s,t] for s in 1:S) == 0
        );
    end
end
for k in 1:K
    for t in shipments[k].departure_time+1:T
        @constraint(
            model_tight,
            sum(x[k,s,t] for s in 1:S) == 0
        );
    end
end

# Each location can hold at most one product at any specific time:
for s in 1:S
    for t in 1:T
        @constraint(
            model_tight,
            sum(x[k,s,t]for k in 1:K) <= 1
        );
    end
end

# Each location can only hold items up to max size of the location
for s in 1:S
    for t in 1:T
        for k in 1:K
            @constraint(
                model_tight,
                x[k,s,t] * shipments[k].product_size <= location_max_size[l]
            );
        end
    end
end


for k in 1:K
    for s in 1:S
            @constraint(
                x[k, l] = sum(x[k,s,t] for l in 1:T) > 0
            )
    end
end

## adding stochasticity

we are designing daily optimization module, which returns $r_{p.l,t}$ for those among the shipments arriving on time t. 

Below function's input is:
- t 
- list of shipments arrived at time t (pa(t)) 
output: 

$$
$pa(t)$ := products that arrived at time t
min \sum_{p \in pa_t} \sum_l d_{al}r_{p,l,t} + \lambda d_{d,l} + \theta_{t+1}
s.t. \sim_{p \in }
$$

In [10]:
findall(x -> (x[1] == 6), shipments)

MethodError: MethodError: no method matching getindex(::Shipment, ::Int64)